In [31]:
devtools::install_github("cellgeni/sceasy")
install.packages('anndata')
library(anndata)

Skipping install of 'sceasy' from a github remote, the SHA1 (0cfc0e39) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/home/jupyter/libb’
(as ‘lib’ is unspecified)



In [32]:
library(sceasy)
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(lme4))
suppressPackageStartupMessages(library(MAST))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(parallel)
library(DESeq2)
options("mc.cores"=50)

In [33]:
Sys.sleep(3)

character(0)

# MAST

In [ ]:
for (files in h5ad_FILES){
pbmc <-sceasy::convertFormat(files, from="anndata", to="seurat")
print(files)
pbmc <- NormalizeData(pbmc, normalization.method = "LogNormalize", scale.factor = 10000)
pbmc@meta.data$CDR <- scale(pbmc@meta.data$nFeature_RNA, scale = TRUE, center = TRUE)
pbmc@meta.data$CMV<-pbmc@meta.data$CMV.IgG.Serology.Result.Interpretation
pbmc@meta.data$Visit<-gsub(" ","_",pbmc@meta.data$sample.visitName)
pbmc@meta.data$Sex<-pbmc@meta.data$subject.biologicalSex
pbmc@meta.data$Donor<-pbmc@meta.data$subject.subjectGuid

i =unique(pbmc@meta.data$Visit)
selCells <- pbmc@meta.data %>%
                      as.data.frame() %>%
                      rownames(.)
esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@data


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]


fdat <- data.frame(rownames(x = normCountssub))
colnames(x = fdat)[1] <- "primerid"
rownames(x = fdat) <- fdat[, 1]


cdat <- esetNormsub2@meta.data %>%
                  as.data.frame() %>%
                  dplyr::select(pbmc_sample_id,Donor,cohort.cohortGuid,
                                CMV,Sex,Visit,CDR)
          cdat <- cdat[colnames(normCountssub), , drop = F]
          table(rownames(cdat) == colnames(normCountssub))
cdat[] <- lapply(cdat, as.character)
cdat$cohort.cohortGuid <- factor(cdat$cohort.cohortGuid)
cdat$cohort.cohortGuid <- relevel(cdat$cohort.cohortGuid, ref = "BR1")


cdat$CDR<-as.numeric(cdat$CDR)
sca <- FromMatrix(exprsArray = as.matrix(normCountssub),
                            cData = cdat,
                            fData = fdat)
zlmCond <- zlm(formula = ~cohort.cohortGuid+Sex+CMV+CDR +(1|Donor),
                         sca = sca,
                         method = "glmer",
                         ebayes = FALSE,
                         parallel = TRUE,
               fitArgsD=list(nAGQ = 0,
                             control = glmerControl(calc.derivs = FALSE)))


summaryCond <- summary(zlmCond, doLRT='cohort.cohortGuidBR2'  ,parallel = TRUE)

summaryDt <- summaryCond$datatable

write.csv(summaryDt,paste0("05B_MAST_BR1_VS_BR2/",as.character(unique(pbmc[[]]$predicted.celltype.l2.5)),'-',i,".csv"))

    
rm(pbmc)    
gc()    
    
}

X -> counts



[1] "05_h5ad_by_celltype//CD14 Mono_Y1D0.h5ad"


`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.


Done!

Combining coefficients and standard errors

Calculating log-fold changes

Calculating likelihood ratio tests

Refitting on reduced model...


Done!

X -> counts



[1] "05_h5ad_by_celltype//CD14 Mono_Y1D90.h5ad"


`cData` has no wellKey.  I'll make something up.

Assuming data assay in position 1, with name et is log-transformed.


Done!

Combining coefficients and standard errors

Calculating log-fold changes

Calculating likelihood ratio tests

Refitting on reduced model...



# Deseq2-BR1 VS BR2 control with CMV, Sex

### L2

In [4]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L2/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L2//ASDC.h5ad"                 
 [2] "01_h5ad_by_AIFI_L2//CD14 monocyte.h5ad"        
 [3] "01_h5ad_by_AIFI_L2//CD16 monocyte.h5ad"        
 [4] "01_h5ad_by_AIFI_L2//CD56bright NK cell.h5ad"   
 [5] "01_h5ad_by_AIFI_L2//CD56dim NK cell.h5ad"      
 [6] "01_h5ad_by_AIFI_L2//CD8aa.h5ad"                
 [7] "01_h5ad_by_AIFI_L2//cDC1.h5ad"                 
 [8] "01_h5ad_by_AIFI_L2//cDC2.h5ad"                 
 [9] "01_h5ad_by_AIFI_L2//DN T cell.h5ad"            
[10] "01_h5ad_by_AIFI_L2//Effector B cell.h5ad"      
[11] "01_h5ad_by_AIFI_L2//Erythrocyte.h5ad"          
[12] "01_h5ad_by_AIFI_L2//gdT.h5ad"                  
[13] "01_h5ad_by_AIFI_L2//ILC.h5ad"                  
[14] "01_h5ad_by_AIFI_L2//Intermediate monocyte.h5ad"
[15] "01_h5ad_by_AIFI_L2//MAIT.h5ad"                 
[16] "01_h5ad_by_AIFI_L2//Memory B cell.h5ad"        
[17] "01_h5ad_by_AIFI_L2//Memory CD4 T cell.h5ad"    
[18] "01_h5ad_by_AIFI_L2//Memory CD8 T cell.h5ad"    
[19] "01_h5ad_by_AIFI_L2//Naive B cell.h5ad"         
[20] "01_h5ad_by_AIFI_L2//Naive CD4 T cell.h5ad"     
[21] "01_h5ad_by_AIFI_L2//Naive CD8 T cell.h5ad"     
[22] "01_h5ad_by_AIFI_L2//pDC.h5ad"                  
[23] "01_h5ad_by_AIFI_L2//Plasma cell.h5ad"          
[24] "01_h5ad_by_AIFI_L2//Platelet.h5ad"             
[25] "01_h5ad_by_AIFI_L2//Progenitor cell.h5ad"      
[26] "01_h5ad_by_AIFI_L2//Proliferating NK cell.h5ad"
[27] "01_h5ad_by_AIFI_L2//Proliferating T cell.h5ad" 
[28] "01_h5ad_by_AIFI_L2//Transitional B cell.h5ad"  
[29] "01_h5ad_by_AIFI_L2//Treg.h5ad"

In [5]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L2))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L2/",as.character(unique(pbmc[[]]$AIFI_L2)),".csv"))
},mc.cores=3)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

### L3

In [6]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L3//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3//ASDC.h5ad"                                
 [4] "01_h5ad_by_AIFI_L3//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3//Core CD14 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3//Core CD16 monocyte.h5ad"                  
[21] "01_h5ad_by_AIFI_L3//Core memory B cell.h5ad"                  
[22] "01_h5ad_by_AIFI_L3//Core naive B cell.h5ad"                   
[23] "01_h5ad_by_AIFI_L3//Core naive CD4 T cell.h5ad"               
[24] "01_h5ad_by_AIFI_L3//Core naive CD8 T cell .h5ad"              
[25] "01_h5ad_by_AIFI_L3//DN T cell.h5ad"                           
[26] "01_h5ad_by_AIFI_L3//Early memory B cell.h5ad"                 
[27] "01_h5ad_by_AIFI_L3//Erythrocyte.h5ad"                         
[28] "01_h5ad_by_AIFI_L3//GZMB- CD27- EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3//GZMB- CD27+ EM CD4 T cell.h5ad"           
[30] "01_h5ad_by_AIFI_L3//GZMB+ Vd2 gdT.h5ad"                       
[31] "01_h5ad_by_AIFI_L3//GZMK- CD27+ EM CD8 T cell.h5ad"           
[32] "01_h5ad_by_AIFI_L3//GZMK- CD56dim NK cell.h5ad"               
[33] "01_h5ad_by_AIFI_L3//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[34] "01_h5ad_by_AIFI_L3//GZMK+ CD56dim NK cell.h5ad"               
[35] "01_h5ad_by_AIFI_L3//GZMK+ memory CD4 Treg.h5ad"               
[36] "01_h5ad_by_AIFI_L3//GZMK+ Vd2 gdT.h5ad"                       
[37] "01_h5ad_by_AIFI_L3//HLA-DRhi cDC2.h5ad"                       
[38] "01_h5ad_by_AIFI_L3//IL1B+ CD14 monocyte.h5ad"                 
[39] "01_h5ad_by_AIFI_L3//ILC.h5ad"                                 
[40] "01_h5ad_by_AIFI_L3//Intermediate monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3//ISG+ CD14 monocyte.h5ad"                  
[42] "01_h5ad_by_AIFI_L3//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3//KLRF1+ GZMB+ CD27- EM CD8 T cell.h5ad"    
[58] "01_h5ad_by_AIFI_L3//Memory CD4 Treg.h5ad"                     

In [7]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3/",as.character(unique(pbmc[[]]$AIFI_L3)),".csv"))
},mc.cores=3)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

### L3.5

In [8]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3.5/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES


[1] "01_h5ad_by_AIFI_L3.5//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3.5//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3.5//AP1hi CD14 monocyte.h5ad"                 
 [4] "01_h5ad_by_AIFI_L3.5//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3.5//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3.5//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3.5//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3.5//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3.5//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3.5//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3.5//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3.5//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3.5//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3.5//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3.5//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3.5//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3.5//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3.5//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3.5//Core CD16 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3.5//Core memory B cell.h5ad"                  
[21] "01_h5ad_by_AIFI_L3.5//Core naive B cell.h5ad"                   
[22] "01_h5ad_by_AIFI_L3.5//Core naive CD4 T cell.h5ad"               
[23] "01_h5ad_by_AIFI_L3.5//Core naive CD8 T cell .h5ad"              
[24] "01_h5ad_by_AIFI_L3.5//DN T cell.h5ad"                           
[25] "01_h5ad_by_AIFI_L3.5//Early memory B cell.h5ad"                 
[26] "01_h5ad_by_AIFI_L3.5//Erythrocyte.h5ad"                         
[27] "01_h5ad_by_AIFI_L3.5//GZMB- CD27- EM CD4 T cell.h5ad"           
[28] "01_h5ad_by_AIFI_L3.5//GZMB- CD27+ EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3.5//GZMB+ Vd2 gdT.h5ad"                       
[30] "01_h5ad_by_AIFI_L3.5//GZMK- CD27+ EM CD8 T cell.h5ad"           
[31] "01_h5ad_by_AIFI_L3.5//GZMK- CD56dim NK cell.h5ad"               
[32] "01_h5ad_by_AIFI_L3.5//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[33] "01_h5ad_by_AIFI_L3.5//GZMK+ CD56dim NK cell.h5ad"               
[34] "01_h5ad_by_AIFI_L3.5//GZMK+ memory CD4 Treg.h5ad"               
[35] "01_h5ad_by_AIFI_L3.5//GZMK+ Vd2 gdT.h5ad"                       
[36] "01_h5ad_by_AIFI_L3.5//HLA-DRhi cDC2.h5ad"                       
[37] "01_h5ad_by_AIFI_L3.5//IL1B+ CD14 monocyte.h5ad"                 
[38] "01_h5ad_by_AIFI_L3.5//ILC.h5ad"                                 
[39] "01_h5ad_by_AIFI_L3.5//Intermediate monocyte.h5ad"               
[40] "01_h5ad_by_AIFI_L3.5//ISG Ihi CD14 monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3.5//ISG IIhi CD14 monocyte.h5ad"              
[42] "01_h5ad_by_AIFI_L3.5//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3.5//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3.5//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3.5//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3.5//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3.5//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3.5//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3.

In [9]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3.5))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3.5/",as.character(unique(pbmc[[]]$AIFI_L3.5)),".csv"))
},mc.cores=3)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

# Deseq2-CMV- VS CMV+ control with Age, Sex

### L2

In [14]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L2/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L2//ASDC.h5ad"                 
 [2] "01_h5ad_by_AIFI_L2//CD14 monocyte.h5ad"        
 [3] "01_h5ad_by_AIFI_L2//CD16 monocyte.h5ad"        
 [4] "01_h5ad_by_AIFI_L2//CD56bright NK cell.h5ad"   
 [5] "01_h5ad_by_AIFI_L2//CD56dim NK cell.h5ad"      
 [6] "01_h5ad_by_AIFI_L2//CD8aa.h5ad"                
 [7] "01_h5ad_by_AIFI_L2//cDC1.h5ad"                 
 [8] "01_h5ad_by_AIFI_L2//cDC2.h5ad"                 
 [9] "01_h5ad_by_AIFI_L2//DN T cell.h5ad"            
[10] "01_h5ad_by_AIFI_L2//Effector B cell.h5ad"      
[11] "01_h5ad_by_AIFI_L2//Erythrocyte.h5ad"          
[12] "01_h5ad_by_AIFI_L2//gdT.h5ad"                  
[13] "01_h5ad_by_AIFI_L2//ILC.h5ad"                  
[14] "01_h5ad_by_AIFI_L2//Intermediate monocyte.h5ad"
[15] "01_h5ad_by_AIFI_L2//MAIT.h5ad"                 
[16] "01_h5ad_by_AIFI_L2//Memory B cell.h5ad"        
[17] "01_h5ad_by_AIFI_L2//Memory CD4 T cell.h5ad"    
[18] "01_h5ad_by_AIFI_L2//Memory CD8 T cell.h5ad"    
[19] "01_h5ad_by_AIFI_L2//Naive B cell.h5ad"         
[20] "01_h5ad_by_AIFI_L2//Naive CD4 T cell.h5ad"     
[21] "01_h5ad_by_AIFI_L2//Naive CD8 T cell.h5ad"     
[22] "01_h5ad_by_AIFI_L2//pDC.h5ad"                  
[23] "01_h5ad_by_AIFI_L2//Plasma cell.h5ad"          
[24] "01_h5ad_by_AIFI_L2//Platelet.h5ad"             
[25] "01_h5ad_by_AIFI_L2//Progenitor cell.h5ad"      
[26] "01_h5ad_by_AIFI_L2//Proliferating NK cell.h5ad"
[27] "01_h5ad_by_AIFI_L2//Proliferating T cell.h5ad" 
[28] "01_h5ad_by_AIFI_L2//Transitional B cell.h5ad"  
[29] "01_h5ad_by_AIFI_L2//Treg.h5ad"

In [15]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)


res <- data.frame(results(dds,contrast=c("CMV.IgG.Serology.Result.Interpretation", "Positive", "Negative")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L2))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInNegative',log2FoldChange>0~'HigherInPositive'))
write.csv(res,paste0("02B_DESeq2_CMVpos_VS_CMVneg_L2/",as.character(unique(pbmc[[]]$AIFI_L2)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

### L3

In [17]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L3//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3//ASDC.h5ad"                                
 [4] "01_h5ad_by_AIFI_L3//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3//Core CD14 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3//Core CD16 monocyte.h5ad"                  
[21] "01_h5ad_by_AIFI_L3//Core memory B cell.h5ad"                  
[22] "01_h5ad_by_AIFI_L3//Core naive B cell.h5ad"                   
[23] "01_h5ad_by_AIFI_L3//Core naive CD4 T cell.h5ad"               
[24] "01_h5ad_by_AIFI_L3//Core naive CD8 T cell .h5ad"              
[25] "01_h5ad_by_AIFI_L3//DN T cell.h5ad"                           
[26] "01_h5ad_by_AIFI_L3//Early memory B cell.h5ad"                 
[27] "01_h5ad_by_AIFI_L3//Erythrocyte.h5ad"                         
[28] "01_h5ad_by_AIFI_L3//GZMB- CD27- EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3//GZMB- CD27+ EM CD4 T cell.h5ad"           
[30] "01_h5ad_by_AIFI_L3//GZMB+ Vd2 gdT.h5ad"                       
[31] "01_h5ad_by_AIFI_L3//GZMK- CD27+ EM CD8 T cell.h5ad"           
[32] "01_h5ad_by_AIFI_L3//GZMK- CD56dim NK cell.h5ad"               
[33] "01_h5ad_by_AIFI_L3//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[34] "01_h5ad_by_AIFI_L3//GZMK+ CD56dim NK cell.h5ad"               
[35] "01_h5ad_by_AIFI_L3//GZMK+ memory CD4 Treg.h5ad"               
[36] "01_h5ad_by_AIFI_L3//GZMK+ Vd2 gdT.h5ad"                       
[37] "01_h5ad_by_AIFI_L3//HLA-DRhi cDC2.h5ad"                       
[38] "01_h5ad_by_AIFI_L3//IL1B+ CD14 monocyte.h5ad"                 
[39] "01_h5ad_by_AIFI_L3//ILC.h5ad"                                 
[40] "01_h5ad_by_AIFI_L3//Intermediate monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3//ISG+ CD14 monocyte.h5ad"                  
[42] "01_h5ad_by_AIFI_L3//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3//KLRF1+ GZMB+ CD27- EM CD8 T cell.h5ad"    
[58] "01_h5ad_by_AIFI_L3//Memory CD4 Treg.h5ad"                     

In [18]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)


res <- data.frame(results(dds,contrast=c("CMV.IgG.Serology.Result.Interpretation", "Positive", "Negative")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInNegative',log2FoldChange>0~'HigherInPositive'))
write.csv(res,paste0("02B_DESeq2_CMVpos_VS_CMVneg_L3/",as.character(unique(pbmc[[]]$AIFI_L3)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

### L3.5

In [19]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3.5/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L3.5//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3.5//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3.5//AP1hi CD14 monocyte.h5ad"                 
 [4] "01_h5ad_by_AIFI_L3.5//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3.5//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3.5//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3.5//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3.5//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3.5//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3.5//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3.5//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3.5//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3.5//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3.5//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3.5//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3.5//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3.5//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3.5//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3.5//Core CD16 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3.5//Core memory B cell.h5ad"                  
[21] "01_h5ad_by_AIFI_L3.5//Core naive B cell.h5ad"                   
[22] "01_h5ad_by_AIFI_L3.5//Core naive CD4 T cell.h5ad"               
[23] "01_h5ad_by_AIFI_L3.5//Core naive CD8 T cell .h5ad"              
[24] "01_h5ad_by_AIFI_L3.5//DN T cell.h5ad"                           
[25] "01_h5ad_by_AIFI_L3.5//Early memory B cell.h5ad"                 
[26] "01_h5ad_by_AIFI_L3.5//Erythrocyte.h5ad"                         
[27] "01_h5ad_by_AIFI_L3.5//GZMB- CD27- EM CD4 T cell.h5ad"           
[28] "01_h5ad_by_AIFI_L3.5//GZMB- CD27+ EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3.5//GZMB+ Vd2 gdT.h5ad"                       
[30] "01_h5ad_by_AIFI_L3.5//GZMK- CD27+ EM CD8 T cell.h5ad"           
[31] "01_h5ad_by_AIFI_L3.5//GZMK- CD56dim NK cell.h5ad"               
[32] "01_h5ad_by_AIFI_L3.5//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[33] "01_h5ad_by_AIFI_L3.5//GZMK+ CD56dim NK cell.h5ad"               
[34] "01_h5ad_by_AIFI_L3.5//GZMK+ memory CD4 Treg.h5ad"               
[35] "01_h5ad_by_AIFI_L3.5//GZMK+ Vd2 gdT.h5ad"                       
[36] "01_h5ad_by_AIFI_L3.5//HLA-DRhi cDC2.h5ad"                       
[37] "01_h5ad_by_AIFI_L3.5//IL1B+ CD14 monocyte.h5ad"                 
[38] "01_h5ad_by_AIFI_L3.5//ILC.h5ad"                                 
[39] "01_h5ad_by_AIFI_L3.5//Intermediate monocyte.h5ad"               
[40] "01_h5ad_by_AIFI_L3.5//ISG Ihi CD14 monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3.5//ISG IIhi CD14 monocyte.h5ad"              
[42] "01_h5ad_by_AIFI_L3.5//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3.5//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3.5//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3.5//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3.5//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3.5//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3.5//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3.

In [20]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)

dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("CMV.IgG.Serology.Result.Interpretation", "Positive", "Negative")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3.5))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInNegative',log2FoldChange>0~'HigherInPositive'))
write.csv(res,paste0("02B_DESeq2_CMVpos_VS_CMVneg_L3.5/",as.character(unique(pbmc[[]]$AIFI_L3.5)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

# Deseq2-BR1 VS BR2 control Sex- seperate CMV+ vs CMV-

### L2

In [34]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L2/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L2//ASDC.h5ad"                 
 [2] "01_h5ad_by_AIFI_L2//CD14 monocyte.h5ad"        
 [3] "01_h5ad_by_AIFI_L2//CD16 monocyte.h5ad"        
 [4] "01_h5ad_by_AIFI_L2//CD56bright NK cell.h5ad"   
 [5] "01_h5ad_by_AIFI_L2//CD56dim NK cell.h5ad"      
 [6] "01_h5ad_by_AIFI_L2//CD8aa.h5ad"                
 [7] "01_h5ad_by_AIFI_L2//cDC1.h5ad"                 
 [8] "01_h5ad_by_AIFI_L2//cDC2.h5ad"                 
 [9] "01_h5ad_by_AIFI_L2//DN T cell.h5ad"            
[10] "01_h5ad_by_AIFI_L2//Effector B cell.h5ad"      
[11] "01_h5ad_by_AIFI_L2//Erythrocyte.h5ad"          
[12] "01_h5ad_by_AIFI_L2//gdT.h5ad"                  
[13] "01_h5ad_by_AIFI_L2//ILC.h5ad"                  
[14] "01_h5ad_by_AIFI_L2//Intermediate monocyte.h5ad"
[15] "01_h5ad_by_AIFI_L2//MAIT.h5ad"                 
[16] "01_h5ad_by_AIFI_L2//Memory B cell.h5ad"        
[17] "01_h5ad_by_AIFI_L2//Memory CD4 T cell.h5ad"    
[18] "01_h5ad_by_AIFI_L2//Memory CD8 T cell.h5ad"    
[19] "01_h5ad_by_AIFI_L2//Naive B cell.h5ad"         
[20] "01_h5ad_by_AIFI_L2//Naive CD4 T cell.h5ad"     
[21] "01_h5ad_by_AIFI_L2//Naive CD8 T cell.h5ad"     
[22] "01_h5ad_by_AIFI_L2//pDC.h5ad"                  
[23] "01_h5ad_by_AIFI_L2//Plasma cell.h5ad"          
[24] "01_h5ad_by_AIFI_L2//Platelet.h5ad"             
[25] "01_h5ad_by_AIFI_L2//Progenitor cell.h5ad"      
[26] "01_h5ad_by_AIFI_L2//Proliferating NK cell.h5ad"
[27] "01_h5ad_by_AIFI_L2//Proliferating T cell.h5ad" 
[28] "01_h5ad_by_AIFI_L2//Transitional B cell.h5ad"  
[29] "01_h5ad_by_AIFI_L2//Treg.h5ad"

In [35]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Positive")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L2))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L2_on_CMVpos/",as.character(unique(pbmc[[]]$AIFI_L2)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

In [36]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Negative")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L2))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L2_on_CMVneg/",as.character(unique(pbmc[[]]$AIFI_L2)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

### L3

In [37]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L3//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3//ASDC.h5ad"                                
 [4] "01_h5ad_by_AIFI_L3//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3//Core CD14 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3//Core CD16 monocyte.h5ad"                  
[21] "01_h5ad_by_AIFI_L3//Core memory B cell.h5ad"                  
[22] "01_h5ad_by_AIFI_L3//Core naive B cell.h5ad"                   
[23] "01_h5ad_by_AIFI_L3//Core naive CD4 T cell.h5ad"               
[24] "01_h5ad_by_AIFI_L3//Core naive CD8 T cell .h5ad"              
[25] "01_h5ad_by_AIFI_L3//DN T cell.h5ad"                           
[26] "01_h5ad_by_AIFI_L3//Early memory B cell.h5ad"                 
[27] "01_h5ad_by_AIFI_L3//Erythrocyte.h5ad"                         
[28] "01_h5ad_by_AIFI_L3//GZMB- CD27- EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3//GZMB- CD27+ EM CD4 T cell.h5ad"           
[30] "01_h5ad_by_AIFI_L3//GZMB+ Vd2 gdT.h5ad"                       
[31] "01_h5ad_by_AIFI_L3//GZMK- CD27+ EM CD8 T cell.h5ad"           
[32] "01_h5ad_by_AIFI_L3//GZMK- CD56dim NK cell.h5ad"               
[33] "01_h5ad_by_AIFI_L3//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[34] "01_h5ad_by_AIFI_L3//GZMK+ CD56dim NK cell.h5ad"               
[35] "01_h5ad_by_AIFI_L3//GZMK+ memory CD4 Treg.h5ad"               
[36] "01_h5ad_by_AIFI_L3//GZMK+ Vd2 gdT.h5ad"                       
[37] "01_h5ad_by_AIFI_L3//HLA-DRhi cDC2.h5ad"                       
[38] "01_h5ad_by_AIFI_L3//IL1B+ CD14 monocyte.h5ad"                 
[39] "01_h5ad_by_AIFI_L3//ILC.h5ad"                                 
[40] "01_h5ad_by_AIFI_L3//Intermediate monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3//ISG+ CD14 monocyte.h5ad"                  
[42] "01_h5ad_by_AIFI_L3//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3//KLRF1+ GZMB+ CD27- EM CD8 T cell.h5ad"    
[58] "01_h5ad_by_AIFI_L3//Memory CD4 Treg.h5ad"                     

In [38]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Positive")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3_on_CMVpos/",as.character(unique(pbmc[[]]$AIFI_L3)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

In [39]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Negative")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3_on_CMVneg/",as.character(unique(pbmc[[]]$AIFI_L3)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

### L3.5

In [40]:
h5ad_FILES<-list.files(path = "01_h5ad_by_AIFI_L3.5/", pattern = ".h5ad", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE,no.. = FALSE)
h5ad_FILES

[1] "01_h5ad_by_AIFI_L3.5//Activated memory B cell.h5ad"             
 [2] "01_h5ad_by_AIFI_L3.5//Adaptive NK cell.h5ad"                    
 [3] "01_h5ad_by_AIFI_L3.5//AP1hi CD14 monocyte.h5ad"                 
 [4] "01_h5ad_by_AIFI_L3.5//BaEoMaP cell.h5ad"                        
 [5] "01_h5ad_by_AIFI_L3.5//C1Q+ CD16 monocyte.h5ad"                  
 [6] "01_h5ad_by_AIFI_L3.5//CD14+ cDC2.h5ad"                          
 [7] "01_h5ad_by_AIFI_L3.5//CD27- effector B cell.h5ad"               
 [8] "01_h5ad_by_AIFI_L3.5//CD27+ effector B cell.h5ad"               
 [9] "01_h5ad_by_AIFI_L3.5//CD4 MAIT.h5ad"                            
[10] "01_h5ad_by_AIFI_L3.5//CD56bright NK cell.h5ad"                  
[11] "01_h5ad_by_AIFI_L3.5//CD8 MAIT.h5ad"                            
[12] "01_h5ad_by_AIFI_L3.5//CD8aa.h5ad"                               
[13] "01_h5ad_by_AIFI_L3.5//CD95 memory B cell.h5ad"                  
[14] "01_h5ad_by_AIFI_L3.5//cDC1.h5ad"                                
[15] "01_h5ad_by_AIFI_L3.5//CLP cell.h5ad"                            
[16] "01_h5ad_by_AIFI_L3.5//CM CD4 T cell.h5ad"                       
[17] "01_h5ad_by_AIFI_L3.5//CM CD8 T cell.h5ad"                       
[18] "01_h5ad_by_AIFI_L3.5//CMP cell.h5ad"                            
[19] "01_h5ad_by_AIFI_L3.5//Core CD16 monocyte.h5ad"                  
[20] "01_h5ad_by_AIFI_L3.5//Core memory B cell.h5ad"                  
[21] "01_h5ad_by_AIFI_L3.5//Core naive B cell.h5ad"                   
[22] "01_h5ad_by_AIFI_L3.5//Core naive CD4 T cell.h5ad"               
[23] "01_h5ad_by_AIFI_L3.5//Core naive CD8 T cell .h5ad"              
[24] "01_h5ad_by_AIFI_L3.5//DN T cell.h5ad"                           
[25] "01_h5ad_by_AIFI_L3.5//Early memory B cell.h5ad"                 
[26] "01_h5ad_by_AIFI_L3.5//Erythrocyte.h5ad"                         
[27] "01_h5ad_by_AIFI_L3.5//GZMB- CD27- EM CD4 T cell.h5ad"           
[28] "01_h5ad_by_AIFI_L3.5//GZMB- CD27+ EM CD4 T cell.h5ad"           
[29] "01_h5ad_by_AIFI_L3.5//GZMB+ Vd2 gdT.h5ad"                       
[30] "01_h5ad_by_AIFI_L3.5//GZMK- CD27+ EM CD8 T cell.h5ad"           
[31] "01_h5ad_by_AIFI_L3.5//GZMK- CD56dim NK cell.h5ad"               
[32] "01_h5ad_by_AIFI_L3.5//GZMK+ CD27+ EM CD8 T cell.h5ad"           
[33] "01_h5ad_by_AIFI_L3.5//GZMK+ CD56dim NK cell.h5ad"               
[34] "01_h5ad_by_AIFI_L3.5//GZMK+ memory CD4 Treg.h5ad"               
[35] "01_h5ad_by_AIFI_L3.5//GZMK+ Vd2 gdT.h5ad"                       
[36] "01_h5ad_by_AIFI_L3.5//HLA-DRhi cDC2.h5ad"                       
[37] "01_h5ad_by_AIFI_L3.5//IL1B+ CD14 monocyte.h5ad"                 
[38] "01_h5ad_by_AIFI_L3.5//ILC.h5ad"                                 
[39] "01_h5ad_by_AIFI_L3.5//Intermediate monocyte.h5ad"               
[40] "01_h5ad_by_AIFI_L3.5//ISG Ihi CD14 monocyte.h5ad"               
[41] "01_h5ad_by_AIFI_L3.5//ISG IIhi CD14 monocyte.h5ad"              
[42] "01_h5ad_by_AIFI_L3.5//ISG+ CD16 monocyte.h5ad"                  
[43] "01_h5ad_by_AIFI_L3.5//ISG+ CD56dim NK cell.h5ad"                
[44] "01_h5ad_by_AIFI_L3.5//ISG+ cDC2.h5ad"                           
[45] "01_h5ad_by_AIFI_L3.5//ISG+ MAIT.h5ad"                           
[46] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD4 T cell.h5ad"              
[47] "01_h5ad_by_AIFI_L3.5//ISG+ memory CD8 T cell.h5ad"              
[48] "01_h5ad_by_AIFI_L3.5//ISG+ naive B cell.h5ad"                   
[49] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD4 T cell.h5ad"               
[50] "01_h5ad_by_AIFI_L3.5//ISG+ naive CD8 T cell.h5ad"               
[51] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD4 Treg.h5ad"              
[52] "01_h5ad_by_AIFI_L3.5//KLRB1+ memory CD8 Treg.h5ad"              
[53] "01_h5ad_by_AIFI_L3.5//KLRF1- effector Vd1 gdT.h5ad"             
[54] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- EM CD8 T cell.h5ad"    
[55] "01_h5ad_by_AIFI_L3.5//KLRF1- GZMB+ CD27- memory CD4 T cell.h5ad"
[56] "01_h5ad_by_AIFI_L3.5//KLRF1+ effector Vd1 gdT.h5ad"             
[57] "01_h5ad_by_AIFI_L3.

In [41]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Positive")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3.5))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3.5_on_CMVpos/",as.character(unique(pbmc[[]]$AIFI_L3.5)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

In [42]:
mclapply(h5ad_FILES,function(files){
pbmc <- read_h5ad(files)
pbmc <- CreateSeuratObject(counts = t(pbmc$X), meta.data = pbmc$obs)
pbmc<-subset(pbmc,subset=cohort.cohortGuid!="UP1")
pbmc<-subset(pbmc,subset=CMV.IgG.Serology.Result.Interpretation=="Negative")
    
selCells <- pbmc@meta.data %>%as.data.frame() %>%rownames(.)

esetNormsub2 <- pbmc[, selCells]
normCountssub <- esetNormsub2[["RNA"]]@counts


min_expr = 0.1
selGenes <- data.frame(num_cells_expressed =
                                 rowSums(normCountssub > min_expr)) %>%
                      rownames_to_column(var = "Gene") %>%
                      filter(num_cells_expressed >=
                             min_expr*length(selCells))
# remove ribosomal, mitochondrial, LINCs, orfs
selGenes <-selGenes$Gene[!grepl("^RP|^MT-|^LINC|orf",selGenes$Gene)]
normCountssub <- normCountssub[selGenes, ]
df=AggregateExpression(pbmc,assays = "RNA",features = selGenes,return.seurat = FALSE,group.by = "pbmc_sample_id",slot = "counts")$RNA

meta_data<-unique(pbmc@meta.data[c('pbmc_sample_id',
                                   'CMV.IgG.Serology.Result.Interpretation',
                                   'subject.biologicalSex','batch_id',
                                   'cohort.cohortGuid',
                                   'subject.subjectGuid')])
rownames(meta_data)<-meta_data$pbmc_sample_id
df=df[,rownames(meta_data)]
dds <- DESeqDataSetFromMatrix(df, colData = meta_data, 
                              design = ~  cohort.cohortGuid+
                                          subject.biologicalSex
                                          )
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR1", "BR2")))
res$celltype<-as.character(unique(pbmc[[]]$AIFI_L3.5))
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInBR2',log2FoldChange>0~'HigherInBR1'))
write.csv(res,paste0("02B_DESeq2_BR1_VS_BR2_L3.5_on_CMVneg/",as.character(unique(pbmc[[]]$AIFI_L3.5)),".csv"))
},mc.cores=60)

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL